# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [1]:
## import libraries

!pip3 install torchtext==0.4.0

# Standard libraries
import math
import string
import pandas as pd
import numpy as np
import json

# Machine learning and deep learning libraries
import tensorflow
import keras
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.data import Field, LabelField, Example, Dataset, BucketIterator
from torchtext.data.utils import get_tokenizer

# Natural Language Processing (NLP) libraries
import nltk
import spacy
from nltk.corpus import stopwords

# Visualization
import matplotlib.pyplot as plt

In [2]:
## reading in json files

"""
Description of json files
* [train-claims,dev-claims].json: JSON files for the labelled training and development set; 
* evidence.json: JSON file containing a large number of evidence passages (i.e. the “knowledge source”); 
* dev-claims-baseline.json: JSON file containing predictions of a baseline system on the development set;
"""

## relative file paths

## baseline system - will not be used for any training/evaluation
devClaimsBaselineFile='./data/dev-claims-baseline.json'
## use this for model training
trainClaimsFile='./data/train-claims.json'
## use this set for hyperparameter tuning and evaluation metric 
devClaimsFile='./data/dev-claims.json'
## evidence files need to be downloaded through https://drive.google.com/file/d/1JlUzRufknsHzKzvrEjgw8D3n_IRpjzo6/view?usp=sharing as it is to big to be uploaded to github
evidenceFile='./data/evidence.json'

testFile='./data/test-claims-unlabelled.json'

tokenizer=get_tokenizer("basic_english")


In [3]:
# Load the JSON data
with open(trainClaimsFile, 'r') as file:
    trainClaims=json.load(file)
with open(devClaimsFile, 'r') as file:
    devClaims=json.load(file)
with open(evidenceFile, 'r') as file:
    evidenceData=json.load(file)

## Preprocessing data -- lowercase, tokenize, and stopword removal
stopwords=set(nltk.corpus.stopwords.words('english'))
tokenizer=get_tokenizer('basic_english')
punctuations=string.punctuation

def preprocess(text):
    token=tokenizer(text.lower())
    cleanedTokens=[t for t in token if (t not in stopwords) and (t not in punctuations)]
    return ' '.join(cleanedTokens)

for ids, texts in evidenceData.items():
    evidenceData[ids]=preprocess(texts)

# Function to create DataFrame and merge evidence IDs with Text
def createDF(claims, evidence):
    combinedData=[]
    for claimID, claimText in claims.items():
        # Combine the ID with its corresponding evidences
        evidenceID=claimText['evidences']
        evidenceText=(evidence[i] for i in evidenceID if i in evidence)
        combinedData.append({
            'claim_id': claimID,
            'claim_text': preprocess(claimText['claim_text']),
            'evidence_id': evidenceID,
            'evidence_text': " ".join(evidenceText),
            'claim_label': claimText['claim_label']
        })
    # Create DataFrame
    return pd.DataFrame(combinedData)

# Create CSV Files
trainFullMerged=createDF(trainClaims,evidenceData)
devFullMerged=createDF(devClaims,evidenceData)
trainFullMerged.to_csv("data/trainFullMerged.csv", index=False)
devFullMerged.to_csv("data/devFullMerged.csv", index=False)

# Convert evidence into csv as well
evidenceFinal=pd.DataFrame(list(evidenceData.items()),columns=['evidence_id','evidence_text'])
evidenceFinal.to_csv('data/evidencePreprocessed.csv',index=False)

# Convert unlabelled Data into CSV as well
with open(testFile, 'r') as file:
    testData=json.load(file)
for ids, texts in testData.items():
    claim_text = texts['claim_text']
    testData[ids]=preprocess(claim_text)
testFinal=pd.DataFrame(list(testData.items()), columns=['claim_id', 'claim_text'])
testFinal.to_csv('data/testPreprocessed.csv',index=False)

In [4]:
## use this for model training
trainClaimsFile='./data/trainFullMerged.csv'
## use this set for hyperparameter tuning and evaluation metric 
devClaimsFile='./data/devFullMerged.csv'
## evidence files need to be downloaded through googledrive (https://drive.google.com/file/d/1OyihwdAWfqHIOueCB4bLBkYg4hTN_OKm/view?usp=sharing)
evidenceFile='./data/evidencePreprocessed.csv'
## test unlabelled dataset
testFile='./data/testPreprocessed.csv'

trainDataframe=pd.read_csv(trainClaimsFile)
devDataframe=pd.read_csv(devClaimsFile)
evidenceDataframe=pd.read_csv(evidenceFile)
testDataframe=pd.read_csv(testFile)

trainDataframe['claim_text']=trainDataframe['claim_text']
trainDataframe['evidence_id']=trainDataframe['evidence_id'].astype(str).str.strip('[]').str.strip("'").apply(preprocess)
trainDataframe['combined_evidence']=trainDataframe['evidence_id']+" "+trainDataframe['evidence_text']
trainDataframe['combined_input'] = trainDataframe['claim_text'] + " [SEP] " + trainDataframe['evidence_text'].astype(str).apply(preprocess)

devDataframe['claim_text']=devDataframe['claim_text']
devDataframe['evidence_id']=devDataframe['evidence_id'].astype(str).str.strip('[]').str.strip("'").apply(preprocess)
devDataframe['combined_evidence']=devDataframe['evidence_id'] +" "+ devDataframe['evidence_text']
devDataframe['combined_input'] = devDataframe['claim_text'] + " [SEP] " + devDataframe['evidence_text'].astype(str).apply(preprocess)

evidenceDataframe['combined_evidence']=evidenceDataframe['evidence_id']+" "+evidenceDataframe['evidence_text']
evidenceDataframe['combined_evidence']=evidenceDataframe['combined_evidence'].astype(str).str.strip("'").apply(preprocess)


In [5]:
## Preprocessing data -- lowercase, tokenize, and stopword removal
stopwords=set(nltk.corpus.stopwords.words('english'))
tokenizer=get_tokenizer('basic_english')
punctuations=set(string.punctuation)

def preprocess(text):
    tokens=tokenizer(text)
    cleanedTokens = [token for token in tokens if token not in stopwords and token not in punctuations]
    return cleanedTokens

TEXT = Field(tokenize=preprocess, lower=True, init_token='<sos>', eos_token='<eos>', batch_first=False)
LABEL=LabelField(sequential=False)

def createDatasetEncoderInput(dataframe,textTransform,labelTransform):
    field=[('reviewTextInput',textTransform),('evidenceTextOutput',labelTransform)]
    examples=[]
    for _, row in dataframe.iterrows():
        reviewTextInput=row['combined_input']
        evidenceTextOutput=row['claim_label']
        examples.append(Example.fromlist([reviewTextInput,evidenceTextOutput], field))
    return Dataset(examples,fields=field)

def createDatasetOutput(dataframe,labelTransform):
    field=[('evidenceTextOutput',labelTransform)]
    examples=[]
    for _, row in dataframe.iterrows():
        evidenceTextOutput=row['claim_label']
        examples.append(Example.fromlist([evidenceTextOutput], field))
    return Dataset(examples,fields=field)

def createDatasetTest(dataframe,textTransform):
    field=[('reviewTextInput',textTransform)]
    examples=[]
    for _, row in dataframe.iterrows():
        reviewTextInput=row['combined_input']
        examples.append(Example.fromlist([reviewTextInput], field))
    return Dataset(examples,fields=field)

trainTensor = createDatasetEncoderInput(trainDataframe, TEXT,LABEL)
devTensor = createDatasetEncoderInput(devDataframe, TEXT,LABEL)
#evidenceTensor=createDatasetOutput(evidenceDataframe,LABEL)
#testTensor=createDatasetTest(testDataframe,TEXT)


In [6]:
TEXT.build_vocab(trainTensor)
LABEL.build_vocab(trainTensor)

In [10]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

['SUPPORTS', 'NOT_ENOUGH_INFO', 'REFUTES', 'DISPUTED']


# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

AttributeError: 'NoneType' object has no attribute 'size'

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*